In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
OUTPUT_DIR = "tables/2024_11_01"
PLOT_DIR = f"{OUTPUT_DIR}/plots"
os.makedirs(PLOT_DIR, exist_ok=True)

In [3]:
df_observed = pd.read_parquet("data/cleaned_df.parquet")[["date", "impl_volatility", "time_to_maturity", "moneyness"]]
df_observed["date"] = pd.to_datetime(df_observed["date"])
df_observed.head()

,date,impl_volatility,time_to_maturity,moneyness
0,2000-01-03,0.221215,1.024,0.621540
1,2000-01-03,0.184711,0.296,0.214505
2,2000-01-03,0.202513,0.184,0.473627
3,2000-01-03,0.176439,0.072,0.093244
4,2000-01-03,0.230361,1.024,0.686076


In [4]:
prices = pd.read_csv("data/GSPC.csv").rename(columns={"Date": "date"}).copy()
prices["date"] = pd.to_datetime(prices["date"])
prices["price"] = prices["Adj Close"]
prices["return"] = np.log(prices["Adj Close"]) - np.log(prices["Adj Close"].shift(1))
prices["return"] = prices["return"].fillna(0)
prices = prices[["date", "price", "return"]]
prices.head()

,date,price,return
0,2000-01-03,1455.219971,0.000000
1,2000-01-04,1399.420044,-0.039099
2,2000-01-05,1402.109985,0.001920
3,2000-01-06,1403.449951,0.000955
4,2000-01-07,1441.469971,0.026730


In [5]:
df = pd.read_parquet("data/spx_vol_surface_history_full_data_23.parquet")
df = df.reset_index().rename(columns={"index": "date"})
df = pd.merge(df, prices, on="date")
df.head()

,date,ttm_one_month_moneyness_pt_seven,ttm_one_month_moneyness_pt_eightfive,ttm_one_month_moneyness_pt_one,ttm_one_month_moneyness_pt_oneonefive,ttm_one_month_moneyness_pt_onethree,ttm_three_month_moneyness_pt_seven,ttm_three_month_moneyness_pt_eightfive,ttm_three_month_moneyness_pt_one,ttm_three_month_moneyness_pt_oneonefive,...,ttm_two_year_moneyness_pt_eightfive,ttm_two_year_moneyness_pt_one,ttm_two_year_moneyness_pt_oneonefive,ttm_two_year_moneyness_pt_onethree,r_squared,mean_error,mean_absolute_error,observation,price,return
0,2000-01-03,6.194064e-02,0.312185,0.204530,0.130964,0.000000e+00,0.260440,0.275718,0.221527,0.168832,...,0.271155,0.228732,0.205318,0.190285,0.995027,-2.735137e-06,0.001119,114.0,1455.219971,0.000000
1,2000-01-04,1.363700e-01,0.320018,0.240132,0.144813,5.976089e-17,0.416818,0.300325,0.230710,0.182980,...,0.281639,0.247485,0.215400,0.192983,0.997879,-1.101140e-06,0.001053,114.0,1399.420044,-0.039099
2,2000-01-05,9.418156e-18,0.342766,0.229619,0.144372,2.613037e-17,0.097694,0.295088,0.234404,0.188767,...,0.283072,0.246802,0.217927,0.199251,0.998235,-1.896156e-07,0.001036,110.0,1402.109985,0.001920
3,2000-01-06,1.702120e-16,0.154186,0.186524,0.144133,0.000000e+00,0.038355,0.251156,0.235466,0.164924,...,0.274596,0.244797,0.213474,0.200466,0.980723,1.295322e-05,0.002518,107.0,1403.449951,0.000955
4,2000-01-07,7.347157e-01,0.400736,0.205539,0.125236,4.878086e-17,0.393377,0.303629,0.220231,0.170690,...,0.268464,0.237584,0.212596,0.192267,0.999162,-4.525523e-07,0.001423,128.0,1441.469971,0.026730


In [6]:
df = df.dropna()
print(len(df))

5822


In [7]:
# each day we have 5 by 5 data
# the numpy array is Nxttmxmoneyness
# ttm = [0.08333,0.25,0.5,1,2] years
# moneyness = [0.7,0.85,1,1.15,1.3]
cols_map = {
    "ttm_one_month_moneyness_pt_seven": (0, 0),
    "ttm_one_month_moneyness_pt_eightfive": (0, 1),
    "ttm_one_month_moneyness_pt_one": (0, 2),
    "ttm_one_month_moneyness_pt_oneonefive": (0, 3),
    "ttm_one_month_moneyness_pt_onethree": (0, 4),

    "ttm_three_month_moneyness_pt_seven": (1, 0),
    "ttm_three_month_moneyness_pt_eightfive": (1, 1),
    "ttm_three_month_moneyness_pt_one": (1, 2),
    "ttm_three_month_moneyness_pt_oneonefive": (1, 3),
    "ttm_three_month_moneyness_pt_onethree": (1, 4),

    "ttm_six_month_moneyness_pt_seven": (2, 0),
    "ttm_six_month_moneyness_pt_eightfive": (2, 1),
    "ttm_six_month_moneyness_pt_one": (2, 2),
    "ttm_six_month_moneyness_pt_oneonefive": (2, 3),
    "ttm_six_month_moneyness_pt_onethree": (2, 4),

    "ttm_one_year_moneyness_pt_seven": (3, 0),
    "ttm_one_year_moneyness_pt_eightfive": (3, 1),
    "ttm_one_year_moneyness_pt_one": (3, 2),
    "ttm_one_year_moneyness_pt_oneonefive": (3, 3),
    "ttm_one_year_moneyness_pt_onethree": (3, 4),   

    "ttm_two_year_moneyness_pt_seven": (4, 0),
    "ttm_two_year_moneyness_pt_eightfive": (4, 1),
    "ttm_two_year_moneyness_pt_one": (4, 2),
    "ttm_two_year_moneyness_pt_oneonefive": (4, 3),
    "ttm_two_year_moneyness_pt_onethree": (4, 4),
}
ttm_list = [1/12, 3/12, 6/12, 1, 2]
moneyness_list = [0.7, 0.85, 1, 1.15, 1.3]
surface_arr = np.zeros((len(df), 5, 5))

In [8]:
def get_closest(df_observed: pd.DataFrame, subset: pd.DataFrame, ttm, moneyness):
    # Merge the subset with the observed DataFrame to only keep matching dates, also cap the values
    merged = subset.merge(df_observed, on="date")
    merged = merged.loc[(merged["impl_volatility"] >= 0.01) & (merged["impl_volatility"] <= 1.0)]
    
    # Calculate differences in time to maturity and moneyness
    merged["time_to_maturity_diff"] = np.abs(merged["time_to_maturity"] - ttm)
    merged["moneyness_diff"] = np.abs(merged["moneyness"] - moneyness)

    # Rank by the distance and get the closest match
    merged = merged.sort_values(by=["time_to_maturity_diff", "moneyness_diff"])
    # Get the closest match for each date
    closest_df = merged.loc[merged.groupby(["date"]).head(1).index][
        ["date", "time_to_maturity", "moneyness", "impl_volatility"]
    ].sort_values(by=["date"]).reset_index(drop=True)
    
    # merged["rank"] = merged[["time_to_maturity_diff", "moneyness_diff"]].sum(axis=1)
    
    # closest_df = merged.loc[merged.groupby(["date"])["rank"].idxmin()][
    #     ["date", "time_to_maturity", "moneyness", "impl_volatility"]
    # ].reset_index(drop=True)
    
    return closest_df

for (col, idx) in cols_map.items():
    print(col)
    ttm = ttm_list[idx[0]]
    moneyness = moneyness_list[idx[1]]
    # clean up values below 0.01
    condition = (df[col] <= 0.01)
    zero_subset = df[condition]
    closest_df = get_closest(df_observed, zero_subset, ttm, moneyness)
    # match the date and fill in the value
    for index, row in closest_df.iterrows():
        # Update only for matching dates and where the condition is true
        if row["date"] in df["date"].values:
            df.loc[(df["date"] == row["date"]) & condition, col] = row["impl_volatility"]
    
    # clean up values above 1.0
    condition = (df[col] >= 1.0)
    top_subset = df[condition]
    closest_df = get_closest(df_observed, top_subset, ttm, moneyness)
    # match the date and fill in the value
    for index, row in closest_df.iterrows():
        # Update only for matching dates and where the condition is true
        if row["date"] in df["date"].values and all(df.loc[(df["date"] == row["date"]) & condition, col] > row["impl_volatility"]):
            df.loc[(df["date"] == row["date"]) & condition, col] = row["impl_volatility"]
    surface_arr[:, idx[0], idx[1]] = df[col].values

ttm_one_month_moneyness_pt_seven
ttm_one_month_moneyness_pt_eightfive
ttm_one_month_moneyness_pt_one
ttm_one_month_moneyness_pt_oneonefive
ttm_one_month_moneyness_pt_onethree
ttm_three_month_moneyness_pt_seven
ttm_three_month_moneyness_pt_eightfive
ttm_three_month_moneyness_pt_one
ttm_three_month_moneyness_pt_oneonefive
ttm_three_month_moneyness_pt_onethree
ttm_six_month_moneyness_pt_seven
ttm_six_month_moneyness_pt_eightfive
ttm_six_month_moneyness_pt_one
ttm_six_month_moneyness_pt_oneonefive
ttm_six_month_moneyness_pt_onethree
ttm_one_year_moneyness_pt_seven
ttm_one_year_moneyness_pt_eightfive
ttm_one_year_moneyness_pt_one
ttm_one_year_moneyness_pt_oneonefive
ttm_one_year_moneyness_pt_onethree
ttm_two_year_moneyness_pt_seven
ttm_two_year_moneyness_pt_eightfive
ttm_two_year_moneyness_pt_one
ttm_two_year_moneyness_pt_oneonefive
ttm_two_year_moneyness_pt_onethree


In [9]:
df_stats =  pd.DataFrame(index=cols_map.keys(), columns=["min", "1%", "99%", "max","mean","std"])
for k in cols_map:
    df_stats.loc[k, "min"] = df[k].min()
    df_stats.loc[k, "1%"] = df[k].quantile(0.01)
    df_stats.loc[k, "99%"] = df[k].quantile(0.99)
    df_stats.loc[k, "max"] = df[k].max()
    df_stats.loc[k, "mean"] = df[k].mean()
    df_stats.loc[k, "std"] = df[k].std()
idx_name_map = {
    "ttm_one_month_moneyness_pt_seven": r"iv[1/12, 0.7]",
    "ttm_one_month_moneyness_pt_eightfive": r"iv[1/12, 0.85]",
    "ttm_one_month_moneyness_pt_one": r"iv[1/12, 1.0]",
    "ttm_one_month_moneyness_pt_oneonefive": r"iv[1/12, 1.15]",
    "ttm_one_month_moneyness_pt_onethree": r"iv[1/12, 1.3]",

    "ttm_three_month_moneyness_pt_seven": r"iv[1/4, 0.7]",
    "ttm_three_month_moneyness_pt_eightfive": r"iv[1/4, 0.85]",
    "ttm_three_month_moneyness_pt_one": r"iv[1/4, 1.0]",
    "ttm_three_month_moneyness_pt_oneonefive": r"iv[1/4, 1.15]",
    "ttm_three_month_moneyness_pt_onethree": r"iv[1/4, 1.3]",

    "ttm_six_month_moneyness_pt_seven": r"iv[1/2, 0.7]",
    "ttm_six_month_moneyness_pt_eightfive": r"iv[1/2, 0.85]",
    "ttm_six_month_moneyness_pt_one": r"iv[1/2, 1.0]",
    "ttm_six_month_moneyness_pt_oneonefive": r"iv[1/2, 1.15]",
    "ttm_six_month_moneyness_pt_onethree": r"iv[1/2, 1.3]",

    "ttm_one_year_moneyness_pt_seven": r"iv[1, 0.7]",
    "ttm_one_year_moneyness_pt_eightfive": r"iv[1, 0.85]",
    "ttm_one_year_moneyness_pt_one": r"iv[1, 1.0]",
    "ttm_one_year_moneyness_pt_oneonefive": r"iv[1, 1.15]",
    "ttm_one_year_moneyness_pt_onethree": r"iv[1, 1.3]",   

    "ttm_two_year_moneyness_pt_seven": r"iv[2, 0.7]",
    "ttm_two_year_moneyness_pt_eightfive": r"iv[2, 0.85]",
    "ttm_two_year_moneyness_pt_one": r"iv[2, 1.0]",
    "ttm_two_year_moneyness_pt_oneonefive": r"iv[2, 1.15]",
    "ttm_two_year_moneyness_pt_onethree": r"iv[2, 1.3]",
}
df_stats = df_stats.rename(index=idx_name_map)
df_stats.to_csv("stats.csv")
df_stats_ltx = df_stats.style.format("{:.3f}").to_latex(column_format="l" + "c"*len(df_stats.columns), hrules=True)
with open(f"{OUTPUT_DIR}/stats.tex", "w") as f:
    f.write(df_stats_ltx)

In [10]:
print(f"min {df_observed["impl_volatility"].min()}, max {df_observed["impl_volatility"].max()}")

min 0.037369, max 2.374417


In [11]:
df_observed.loc[df_observed["impl_volatility"].idxmax()]

date                2008-09-29 00:00:00
impl_volatility                2.374417
time_to_maturity                  0.072
moneyness                       0.92561
Name: 422265, dtype: object

In [12]:
print(np.min(surface_arr), np.max(surface_arr))

0.010004553926966074 0.9957201040076749


In [13]:
def plot_surface_grids(df: pd.DataFrame, fn):
    cols = [
        [
            "ttm_one_month_moneyness_pt_seven",
            "ttm_one_month_moneyness_pt_eightfive",
            "ttm_one_month_moneyness_pt_one",
            "ttm_one_month_moneyness_pt_oneonefive",
            "ttm_one_month_moneyness_pt_onethree",
        ],
        [
            "ttm_three_month_moneyness_pt_seven",
            "ttm_three_month_moneyness_pt_eightfive",
            "ttm_three_month_moneyness_pt_one",
            "ttm_three_month_moneyness_pt_oneonefive",
            "ttm_three_month_moneyness_pt_onethree",
        ],
        [
            "ttm_six_month_moneyness_pt_seven",
            "ttm_six_month_moneyness_pt_eightfive",
            "ttm_six_month_moneyness_pt_one",
            "ttm_six_month_moneyness_pt_oneonefive",
            "ttm_six_month_moneyness_pt_onethree",
        ],
        [
            "ttm_one_year_moneyness_pt_seven",
            "ttm_one_year_moneyness_pt_eightfive",
            "ttm_one_year_moneyness_pt_one",
            "ttm_one_year_moneyness_pt_oneonefive",
            "ttm_one_year_moneyness_pt_onethree",
        ],   
        [
            "ttm_two_year_moneyness_pt_seven",
            "ttm_two_year_moneyness_pt_eightfive",
            "ttm_two_year_moneyness_pt_one",
            "ttm_two_year_moneyness_pt_oneonefive",
            "ttm_two_year_moneyness_pt_onethree",
        ],
    ]
    fig, ax = plt.subplots(5, 5, figsize=(30, 30))
    df_cols = ["K/S=0.7", "K/S=0.85", "K/S=1", "K/S=1.15", "K/S=1.3"]
    df_rows = ["1 month", "3 month", "6 month", "1 year", "2 year"]
    for row in range(5):
        for col in range(5):
            curr_grid = cols[row][col]
            ax[row][col].hist(df[curr_grid], bins=20)
            ax[row][col].set_title(f"Mean: {df_rows[row]}, {df_cols[col]}")
            ax[row][col].set_xlabel("IV")
            ax[row][col].set_ylabel("Count")
    plt.tight_layout()
    plt.savefig(fn)
    plt.close()
plot_surface_grids(df, fn=f"{PLOT_DIR}/histograms.jpg")

In [14]:
skews = (surface_arr[:, 3, 1] + surface_arr[:, 3, 3]) / 2 - surface_arr[:, 3, 2]
slopes = surface_arr[:, 4, 2] - surface_arr[:, 1, 2]
levels = surface_arr[:, 3, 2]

In [15]:
ret = np.array(df["return"].values)
prices = np.array(df["price"].values)
print(len(surface_arr))
print(len(ret))
print(len(prices))
print(skews.shape)
print(slopes.shape)
print(levels.shape)

5822
5822
5822
(5822,)
(5822,)
(5822,)


In [16]:
np.savez("data/vol_surface_with_ret.npz", surface=surface_arr, ret=ret, price=prices, slopes=slopes, skews=skews, levels=levels)
df.to_parquet("data/spx_vol_surface_history_full_data_fixed.parquet", index=False)

In [17]:
data = np.load("data/vol_surface_with_ret.npz")
print(data.files)

['surface', 'ret', 'price', 'slopes', 'skews', 'levels']


In [18]:
v = np.concatenate([ret[...,np.newaxis], skews[...,np.newaxis], slopes[...,np.newaxis], levels[...,np.newaxis]], axis=-1)
print(v.shape)

(5822, 4)


In [19]:
print(np.min(ret), np.max(ret))

-0.12765219747281709 0.10957196759533883
